In [1]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
train_df = pd.read_csv('/kaggle/input/datadir-celeb-a/aligned_celeba.txt', sep='\t')
train_df['datadir'] = '/kaggle/input/celeba-dataset/img_align_celeba/img_align_celeba/' + train_df['datadir'].astype(str)

In [3]:
defaults = {
    'epochs': 20,
    'batch_size': 128,
    'fc1_num_neurons': 1024,
    'fc2_num_neurons': 512,
    'fc3_num_neurons': 256,
    'seed': 7,
    'learning_rate': 3e-4,
    'optimizer': 'adam',
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

wandb.init(config=defaults, resume=True, name='Correct FC3', project='CelebA Runs', notes='change FC3 to 256 neurons')
config = wandb.config

# Load images into keras image generator 
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

# Define model
mobile_net_v2 = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(224,224,3),
)
mobile_net_v2.trainable = True

fc1 = tf.keras.layers.Dense(config.fc1_num_neurons,activation=config.hidden_activation)
fc2 = tf.keras.layers.Dense(config.fc2_num_neurons,activation=config.hidden_activation)
fc3 = tf.keras.layers.Dense(config.fc3_num_neurons,activation=config.hidden_activation)
bn1 = tf.keras.layers.BatchNormalization()
bn2 = tf.keras.layers.BatchNormalization()
bn3 = tf.keras.layers.BatchNormalization()
bn4 = tf.keras.layers.BatchNormalization()

model = tf.keras.models.Sequential([
    mobile_net_v2,
    tf.keras.layers.Flatten(),
    bn1,
    fc1,
    bn2,
    fc2,
    bn3,
    fc3,
    bn4,
    tf.keras.layers.Dense(1, activation=config.output_activation),
])
model.summary()

# Compile model 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=config.loss_function,
    metrics=config.metrics,
)

model.fit(
    train_generator,
    shuffle=True,
    epochs=config.epochs,
    callbacks=[WandbCallback()],
)
model.save_weights('model_celeba_correct_fc3.h5') 
# run.finish()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 469, in init
    wi.setup(kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 131, in setup
    login_key = wandb.login(anonymous=anonymous, force=force)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 43, in login
    configured = _login(**kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 197, in _login
    wlogin.prompt_api_key()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 139, in prompt_api_key
    no_create=self._settings.force,
  File "/opt/conda/lib/python3.7/site-packages/wandb/lib/apikey.py", line 141, in prompt_api_key
    key = input_callback(api_ask).strip()
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 834, in getpas

Exception: problem